## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

data = pd.read_csv('./energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

### 시퀀스 데이터 생성 함수 작성

In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# 시퀀스 데이터 생성 함수
# dataX: 시퀀스 데이터를 담은 리스트 (예: [[시퀀스1], [시퀀스2], ...])
# dataY: 각 시퀀스에 대응하는 타겟 값 (예: 발전량)

def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i+seq_length, :-1])
        dataY.append(time_series[i+seq_length, [-1]])
    return np.array(dataX), np.array(dataY)

# 하이퍼파라미터 설정
seq_length = 24  # (1) 예: 24시간의 데이터를 시퀀스로 사용
batch_size = 32  # (2) 배치 크기 설정

# 학습 데이터 생성
# trainX: 24시간의 피처 데이터
# trainY: 그 다음 시간에 대한 발전량 값
trainX, trainY = build_dataset(time_series_df.values, seq_length)  # (3) 데이터프레임의 values를 사용

# 텐서로 변환
trainX_tensor = torch.tensor(trainX, dtype=torch.float32)  # (4) numpy 배열을 텐서로 변환
trainY_tensor = torch.tensor(trainY, dtype=torch.float32)  # (5) numpy 배열을 텐서로 변환

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # (6) 배치 크기 설정


### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [ ]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)

from torch.utils.data import DataLoader, TensorDataset

# Load the dataset
file_path = '/C:\Users\장민지\Downloads/energy_tobigs_question.csv' 
energy_data = pd.read_csv(file_path)

# Preprocess the data
energy_data['time'] = pd.to_datetime(energy_data['time'])
energy_data = energy_data.set_index('time')
energy_data = energy_data.asfreq('H')
energy_data = energy_data.interpolate(method='linear')

# Define feature columns and target
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 
            'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# Extract training data up to '2023-10-14 23:00:00'
training_data = energy_data.loc[:'2023-10-14 23:00:00']

# Sliding window dataset preparation
def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i + seq_length, :-1])
        dataY.append(time_series[i + seq_length, -1])
    return np.array(dataX), np.array(dataY)

seq_length = 24  # 24 hours sequence
trainX, trainY = build_dataset(training_data[features + [target]].values, seq_length)

# Convert to tensors
trainX_tensor = torch.tensor(trainX, dtype=torch.float32)
trainY_tensor = torch.tensor(trainY, dtype=torch.float32)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Model parameters
input_size = len(features)
hidden_size = 64
num_layers = 2
output_size = 1

# Instantiate model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_Y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Predict for 2023-10-15 using the trained model
model.eval()
last_24_hours_data = energy_data.loc['2023-10-14 00:00:00':'2023-10-14 23:00:00'][features].values
predictions = []

# Recursive prediction for the next 24 hours
current_input = torch.tensor(last_24_hours_data, dtype=torch.float32).unsqueeze(0)

for _ in range(24):
    with torch.no_grad():
        next_pred = model(current_input).item()
        predictions.append(next_pred)
        
        # Update the current input with the latest prediction
        next_input = np.append(current_input.squeeze(0).numpy(), np.array([[next_pred] * len(features)]), axis=0)
        current_input = torch.tensor(next_input[-seq_length:, :], dtype=torch.float32).unsqueeze(0)

# Display predictions for 2023-10-15
for hour, pred in enumerate(predictions, start=1):
    print(f"2023-10-15 {hour:02}:00: Predicted Generation: {pred:.2f}")


### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [ ]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=data['time'], y=data['amount'],
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=test_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

### 정답을 적어주세요.

In [ ]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)
Transformer는 모든 입력 토큰 간의 관계를 계산하는 완전한 self-attention 메커니즘을 사용하기 때문에 시계열 데이터처럼 길이가 긴 데이터에 비효율적입니다.
Informer는 이러한 비효율성을 해결하기 위해 ProbSparse Self-Attention을 도입했습니다. 이를 통해 모든 쿼리-키 조합을 계산하는 대신, 정보량이 풍부한 쿼리-키 조합만을 선택하여 계산 복잡도를 크게 줄일 수 있습니다.

### (필수) 모델링 해석

# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.

LSTM
1. 시계열 데이터의 특성: 발전량 데이터는 시간에 따라 변화하는 시계열 데이터이며, 과거의 상태가 미래의 상태에 영향을 미치는 연속성이 있습니다. LSTM(Long Short-Term Memory)은 시계열 데이터에서 장기 의존성을 효과적으로 학습할 수 있는 모델이기 때문에 적합합니다.
  
2. 문맥 정보 유지: 일반적인 순환신경망(RNN)은 시계열이 길어지면 과거 정보가 손실될 수 있지만, LSTM은 게이트 구조(입력 게이트, 망각 게이트, 출력 게이트)를 통해 이전 시점의 중요한 정보를 유지하거나 버릴 수 있습니다. 이를 통해 발전량 예측에 필요한 과거 패턴을 잘 학습할 수 있습니다.